### General import

In [56]:
# IMPORTS

import pandas as pd
import numpy as np
import json
from datetime import datetime
from sklearn.cluster import KMeans

### Export poids (weight.json)

In [57]:
# Define Constants
WEIGHT_DF_COLUMN_DATETIME='date'
WEIGHT_DF_COLUMN_WEIGHT='poids'
WEIGHT_FILE_PATH='../data/poids.csv'

WEIGHT_JSON_FIELD_DATE='dates'
WEIGHT_JSON_FIELD_WEIGHT='values'
WEIGHT_JSON_FILE_PATH='../docs/data/weight.json'

In [58]:
# Loading weight data
df_weight=pd.read_csv(WEIGHT_FILE_PATH, sep=';', parse_dates=[WEIGHT_DF_COLUMN_DATETIME])

In [59]:
# Conversion Poids
df_json=pd.DataFrame()
df_json[WEIGHT_JSON_FIELD_DATE]=df_weight[[WEIGHT_DF_COLUMN_DATETIME]].apply(lambda x : x.iloc[0].strftime("%Y-%m-%d"), axis=1)
df_json[WEIGHT_JSON_FIELD_WEIGHT]=df_weight[[WEIGHT_DF_COLUMN_WEIGHT]]
# replace Nan with None, so it will be replaced by null in json
df_json = df_json.replace(np.nan, None)
#df_json

In [60]:
# OUTPUT 
weight_data_object = {
  WEIGHT_JSON_FIELD_DATE: df_json[WEIGHT_JSON_FIELD_DATE].to_list(),
  WEIGHT_JSON_FIELD_WEIGHT: df_json[WEIGHT_JSON_FIELD_WEIGHT].to_list(),
}

with open(WEIGHT_JSON_FILE_PATH, 'w', encoding='utf-8') as f:
    json.dump(weight_data_object, f, indent=4)

# Print
# data_json = json.dumps(weight_data_object)
# print(data_json)

### Export taille

In [61]:
# Define Constants
SIZE_DF_COLUMN_DATETIME='date'
SIZE_DF_COLUMN_SHOULDER='hauteur_epaule'
SIZE_DF_COLUMN_BACK='longueur_dos'
SIZE_FILE_PATH='../data/size.csv'

SIZE_JSON_FIELD_DATE='dates'
SIZE_JSON_FIELD_SHOULDER='shoulder'
SIZE_JSON_FIELD_BACK='back'
SIZE_JSON_FILE_PATH='../docs/data/size.json'

In [62]:
# Loading weight data
df_size=pd.read_csv(SIZE_FILE_PATH, sep=';', parse_dates=[SIZE_DF_COLUMN_DATETIME])
# OUTPUT 
size_data_object = {
  SIZE_JSON_FIELD_DATE: df_size[[SIZE_DF_COLUMN_DATETIME]].apply(lambda x : x.iloc[0].strftime("%Y-%m-%d"), axis=1).to_list(),
  SIZE_JSON_FIELD_SHOULDER: df_size[SIZE_DF_COLUMN_SHOULDER].to_list(),
  SIZE_JSON_FIELD_BACK: df_size[SIZE_DF_COLUMN_BACK].to_list(),
}

with open(SIZE_JSON_FILE_PATH, 'w', encoding='utf-8') as f:
    json.dump(size_data_object, f, indent=4)

### Export propreté

In [63]:
# Define Constants
POOP_DF_COLUMN_DATE='date'
POOP_DF_COLUMN_DATETIME='datetime'
POOP_DF_COLUMN_HOUR='heure'
POOP_DF_COLUMN_DATETIME_OF_DAY='datetime_ofday'
POOP_DF_COLUMN_NATURE='nature'
POOP_DF_VALUE_NATURE_POOP='caca'
POOP_DF_COLUMN_LOCATION='lieu'
POOP_DF_VALUE_LOCATION_IN='in'
POOP_DF_COLUMN_TIMESTAMP_OF_DAY='timestamp_ofday'
POOP_DF_COLUMN_DAY_OF_WEEK='day_of_week'

POOP_JSON_FIELD_DATE='dates'
POOP_JSON_FIELD_DAY_SUCCESS_RATE='day_success_rate'
POOP_JSON_FIELD_DAY_NB_FAILURE='day_nb_failure'
POOP_JSON_FIELD_NIGHT_NB_FAILURE='night_nb_failure'
POOP_JSON_FILE_PATH='../docs/data/poop.json'

POOPBYHOUR_JSON_FILE_PATH='../docs/data/poop_by_hour.json'
POOPBYHOUR_JSON_FIELD_HOUR='hour'
POOPBYHOUR_JSON_FIELD_DAYOFWEEK='day_of_week'
POOPBYHOUR_JSON_FIELD_GROUP='group'

In [64]:
# Loading poop data
# data consolidation: Computing poop date and hour of day as datetime
df_poop=pd.DataFrame(pd.read_excel('../data/poop.xlsx'))
df_poop[POOP_DF_COLUMN_DATE] = pd.to_datetime(df_poop[POOP_DF_COLUMN_DATE]).dt.strftime('%Y-%m-%d')
df_poop[POOP_DF_COLUMN_DATETIME]=df_poop[[POOP_DF_COLUMN_DATE, POOP_DF_COLUMN_HOUR]].apply(lambda x : datetime.strptime('{}T{}'.format(x.iloc[0],x.iloc[1]), "%Y-%m-%dT%H:%M:%S"), axis=1)
_today=datetime.now().strftime("%Y-%m-%d")
df_poop[POOP_DF_COLUMN_DATETIME_OF_DAY]=df_poop[[POOP_DF_COLUMN_HOUR]].apply(lambda x : datetime.strptime('{}T{}'.format(_today,x.iloc[0]), "%Y-%m-%dT%H:%M:%S"), axis=1)
df_poop[POOP_DF_COLUMN_TIMESTAMP_OF_DAY] = pd.DatetimeIndex(df_poop[POOP_DF_COLUMN_DATETIME_OF_DAY]).asi8
df_poop[POOP_DF_COLUMN_DAY_OF_WEEK]= pd.DatetimeIndex(df_poop[POOP_DF_COLUMN_DATETIME]).dayofweek

In [65]:
# filtering during the day data
_start_of_day=datetime.now().replace(hour=6, minute=30, second=0)
_end_of_day=datetime.now().replace(hour=21, minute=0, second=0)

df_poop_day=df_poop[df_poop.apply(lambda x: (x[POOP_DF_COLUMN_DATETIME_OF_DAY] > _start_of_day) & (x[POOP_DF_COLUMN_DATETIME_OF_DAY] < _end_of_day), axis=1)]
df_poop_day_efficiency=df_poop_day.groupby(POOP_DF_COLUMN_DATE).size().to_frame('total').reset_index()
# formule compliquée mais qui permet de compter les jours à zero
df_poop_day_efficiency['success']=df_poop_day.groupby([POOP_DF_COLUMN_DATE, POOP_DF_COLUMN_LOCATION]).size().to_frame('success').unstack(fill_value=0).stack()\
    .query(POOP_DF_COLUMN_LOCATION + ' !=  "' + POOP_DF_VALUE_LOCATION_IN + '"')\
    .groupby(POOP_DF_COLUMN_DATE).sum().reset_index()['success']
df_poop_day_efficiency[POOP_JSON_FIELD_DAY_SUCCESS_RATE]=round(df_poop_day_efficiency['success'] * 100 / df_poop_day_efficiency['total'], 1)
df_poop_day_efficiency[POOP_JSON_FIELD_DAY_NB_FAILURE]=df_poop_day_efficiency['total'] - df_poop_day_efficiency['success']

df_poop_night=df_poop[df_poop.apply(lambda x: (x[POOP_DF_COLUMN_DATETIME_OF_DAY] <= _start_of_day) | (x[POOP_DF_COLUMN_DATETIME_OF_DAY] >= _end_of_day), axis=1)]
df_poop_night_efficiency=df_poop_night.groupby(POOP_DF_COLUMN_DATE).size().to_frame('total').reset_index()
# formule compliquée mais qui permet de compter les jours à zero
df_poop_night_efficiency[POOP_JSON_FIELD_NIGHT_NB_FAILURE]=df_poop_night.groupby([POOP_DF_COLUMN_DATE, POOP_DF_COLUMN_LOCATION]).size().to_frame(POOP_JSON_FIELD_NIGHT_NB_FAILURE).unstack(fill_value=0).stack()\
    .query(POOP_DF_COLUMN_LOCATION + ' ==  "' + POOP_DF_VALUE_LOCATION_IN + '"')\
    .reset_index()[POOP_JSON_FIELD_NIGHT_NB_FAILURE]


C:\Users\julie\AppData\Local\Temp\ipykernel_19296\1827400457.py:8: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_poop_day_efficiency['success']=df_poop_day.groupby([POOP_DF_COLUMN_DATE, POOP_DF_COLUMN_LOCATION]).size().to_frame('success').unstack(fill_value=0).stack()\
C:\Users\julie\AppData\Local\Temp\ipykernel_19296\1827400457.py:17: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_poop_night_efficiency[POOP_JSON_FIELD_NIGHT_NB_FAILURE]=df_poop_night.groupby([POOP_DF_COLUMN_DATE, POOP_DF_COLUMN_LOCATION]).size().to_frame(POOP_JSON_FIELD_NIGHT_NB_FAILURE).unstack(fi

In [66]:
# to Json
# OUTPUT 
poop_data_object = {
  POOP_JSON_FIELD_DATE: df_poop_day_efficiency[POOP_DF_COLUMN_DATE].to_list(),
  POOP_JSON_FIELD_DAY_SUCCESS_RATE: df_poop_day_efficiency[POOP_JSON_FIELD_DAY_SUCCESS_RATE].to_list(),
  POOP_JSON_FIELD_DAY_NB_FAILURE: df_poop_day_efficiency[POOP_JSON_FIELD_DAY_NB_FAILURE].to_list(),
  POOP_JSON_FIELD_NIGHT_NB_FAILURE: df_poop_night_efficiency[POOP_JSON_FIELD_NIGHT_NB_FAILURE].to_list(),
}

with open(POOP_JSON_FILE_PATH, 'w', encoding='utf-8') as f:
    json.dump(poop_data_object, f, indent=4)

# Print
#data_json = json.dumps(data_object)
#print(data_json)

### Export poop by hour of day

In [67]:
# GRAPH poop by hour of day
# filter on poop oly
df_poop_only=df_poop[df_poop.apply(lambda x: (x[POOP_DF_COLUMN_NATURE] == POOP_DF_VALUE_NATURE_POOP) & (x[POOP_DF_COLUMN_DATE] >= '2024-04-05'), axis=1)]
df_poop_group_by_date=df_poop_only.groupby(POOP_DF_COLUMN_DATE).size().to_frame('total').reset_index()
avg_nb_poop_per_day=df_poop_group_by_date['total'].mean()

print(f"Avg Nb poop per day: {round(avg_nb_poop_per_day,0)} ({round(avg_nb_poop_per_day,3)})")

Avg Nb poop per day: 6.0 (6.118)


In [100]:
#Kmeans on poop
kmeans = KMeans(n_clusters=round(avg_nb_poop_per_day), random_state=0)
kmeans.fit(df_poop_only[POOP_DF_COLUMN_TIMESTAMP_OF_DAY].array.reshape(-1,1))

df_poop_group_kmean=pd.DataFrame({\
    POOPBYHOUR_JSON_FIELD_HOUR:df_poop_only[POOP_DF_COLUMN_DATETIME_OF_DAY],\
    POOPBYHOUR_JSON_FIELD_DAYOFWEEK:df_poop_only[POOP_DF_COLUMN_DAY_OF_WEEK],\
    POOPBYHOUR_JSON_FIELD_GROUP:kmeans.labels_})

# Avg value by group
df_poop_group_avg=(df_poop_group_kmean.groupby(POOPBYHOUR_JSON_FIELD_GROUP).mean().sort_values(by=POOPBYHOUR_JSON_FIELD_HOUR))

# Json nexport
poop_by_hour_json=json.loads('{}')
poop_by_hour_json['by_hour_of_day']=json.loads(df_poop_group_kmean.to_json(orient='records', date_format='iso'))
poop_by_hour_json['avg_by_day']=round(avg_nb_poop_per_day,0)
poop_by_hour_json['prefered_hours']=df_poop_group_avg[POOPBYHOUR_JSON_FIELD_HOUR].dt.strftime('%Hh%M').to_list()

with open(POOPBYHOUR_JSON_FILE_PATH, 'w', encoding='utf-8') as f:
    json.dump(poop_by_hour_json, f, indent=4)